In [24]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.ml.feature.CountVectorizerModel

In [2]:
val spark = SparkSession.builder().appName("lab06che").getOrCreate()

spark = org.apache.spark.sql.SparkSession@52aedc57


org.apache.spark.sql.SparkSession@52aedc57

In [3]:
import spark.implicits._

In [4]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

In [5]:
val asArray = udf((v: org.apache.spark.ml.linalg.Vector) => v.toDense.toArray)
val outputPath = "/user/alexey.chernyaev2/features"
val wind = Window.partitionBy('uid)

val dfLogs = spark.read.json("hdfs:///labs/laba03/weblogs.json")
val dfUsersItems = spark.read.parquet("/user/alexey.chernyaev2/users-items/20200429")

asArray = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
outputPath = /user/alexey.chernyaev2/features
wind = org.apache.spark.sql.expressions.WindowSpec@3e640538
dfLogs = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]
dfUsersItems = [uid: string, buy_cameras_0: bigint ... 639 more fields]


[uid: string, buy_cameras_0: bigint ... 639 more fields]

In [6]:
dfUsersItems.show(2,200,true)

-RECORD 0-----------------------------------------------------------------
 uid                               | dd55d458-37fb-461d-81d6-ab01bbf9b82c 
 buy_cameras_0                     | 0                                    
 buy_cameras_1                     | 0                                    
 buy_cameras_10                    | 0                                    
 buy_cameras_11                    | 0                                    
 buy_cameras_12                    | 0                                    
 buy_cameras_13                    | 0                                    
 buy_cameras_14                    | 0                                    
 buy_cameras_15                    | 0                                    
 buy_cameras_16                    | 0                                    
 buy_cameras_17                    | 0                                    
 buy_cameras_18                    | 0                                    
 buy_cameras_19          

In [7]:
dfLogs.filter($"uid" like "%4ae%").show(2)

+--------------------+--------------------+
|                 uid|              visits|
+--------------------+--------------------+
|d50192e5-c44e-4ae...|[[1419688144068, ...|
|d60c4ae2-9244-480...|[[1427176994000, ...|
+--------------------+--------------------+
only showing top 2 rows



In [8]:
val dfLogsDomain = dfLogs.select($"uid", explode($"visits").as("visit"))
                .withColumn("host", lower(callUDF("parse_url", $"visit.url", lit("HOST"))))
                .withColumn("domain", regexp_replace($"host", "www.", ""))
                .withColumn("timestamp", $"visit.timestamp" / 1000)
                .withColumn("hour", from_unixtime($"timestamp", "H"))
                .withColumn("dayOfWeek", concat(lit("web_day_"), lower(date_format($"timestamp".cast("timestamp"),"E"))))
                .filter($"domain".isNotNull)
                .filter($"uid".isNotNull)
                .select("uid", "domain", "dayOfWeek", "hour")
               
dfLogsDomain.show(2,200,true)

-RECORD 0-----------------------------------------
 uid       | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777 
 domain    | zebra-zoya.ru                        
 dayOfWeek | web_day_sat                          
 hour      | 13                                   
-RECORD 1-----------------------------------------
 uid       | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777 
 domain    | news.yandex.ru                       
 dayOfWeek | web_day_wed                          
 hour      | 8                                    
only showing top 2 rows



dfLogsDomain = [uid: string, domain: string ... 2 more fields]


[uid: string, domain: string ... 2 more fields]

In [9]:
val dfWorkOrNot = dfLogsDomain.withColumn("allVisits", count($"domain").over(wind))
                                .withColumn("web_fraction_work_hours", sum(when($"hour" >= lit(9) && $"hour" < lit(18), 1).otherwise(0)).over(wind) / $"allVisits")
                                .withColumn("web_fraction_evening_hours", sum(when($"hour" >= lit(18) && $"hour" < lit(24), 1).otherwise(0)).over(wind) / $"allVisits")
                                .select("uid", "web_fraction_work_hours", "web_fraction_evening_hours")
                                

dfWorkOrNot = [uid: string, web_fraction_work_hours: double ... 1 more field]


[uid: string, web_fraction_work_hours: double ... 1 more field]

In [10]:
dfWorkOrNot.show(2,200,true)

-RECORD 0----------------------------------------------------------
 uid                        | 0392f398-ea7e-4a1b-8c9d-d04df2a2a331 
 web_fraction_work_hours    | 0.17142857142857143                  
 web_fraction_evening_hours | 0.6904761904761905                   
-RECORD 1----------------------------------------------------------
 uid                        | 0392f398-ea7e-4a1b-8c9d-d04df2a2a331 
 web_fraction_work_hours    | 0.17142857142857143                  
 web_fraction_evening_hours | 0.6904761904761905                   
only showing top 2 rows



In [11]:
val dfDayOfWeek = dfLogsDomain.groupBy($"uid").pivot($"dayOfWeek").agg(count($"domain").as("cnt_domain")).na.fill(0)

dfDayOfWeek = [uid: string, web_day_fri: bigint ... 6 more fields]


[uid: string, web_day_fri: bigint ... 6 more fields]

In [12]:
dfDayOfWeek.show(5)

+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|                 uid|web_day_fri|web_day_mon|web_day_sat|web_day_sun|web_day_thu|web_day_tue|web_day_wed|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|7e167ad3-6fc3-4f9...|        214|        405|        291|        373|        298|        184|        227|
|988f6d0a-4a29-45e...|          0|          1|          1|          4|          0|          0|          0|
|2c709e22-c8ee-459...|          4|          3|          0|          0|          2|          6|          2|
|99570b7a-7309-407...|          6|          4|         31|          4|         10|          9|         13|
|b75a3b4e-031c-402...|          1|          1|          3|          1|          1|          1|          0|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
only showing top 5 rows



In [13]:
val dfHour = dfLogsDomain
                .withColumn("hour", concat(lit("web_hour_"), $"hour"))
                .groupBy("uid")
                .pivot("hour").agg(count("domain").as("cnt_domain"))
                .na.fill(0)
dfHour.show(2,200,true)

-RECORD 0-------------------------------------------
 uid         | c1bf0d2b-6e76-4633-b058-e86b9d02a0c2 
 web_hour_0  | 0                                    
 web_hour_1  | 0                                    
 web_hour_10 | 250                                  
 web_hour_11 | 13                                   
 web_hour_12 | 59                                   
 web_hour_13 | 77                                   
 web_hour_14 | 103                                  
 web_hour_15 | 126                                  
 web_hour_16 | 51                                   
 web_hour_17 | 106                                  
 web_hour_18 | 159                                  
 web_hour_19 | 181                                  
 web_hour_2  | 18                                   
 web_hour_20 | 9                                    
 web_hour_21 | 0                                    
 web_hour_22 | 0                                    
 web_hour_23 | 0                              

dfHour = [uid: string, web_hour_0: bigint ... 23 more fields]


[uid: string, web_hour_0: bigint ... 23 more fields]

In [25]:
val Vocabulary = dfLogsDomain.groupBy("domain")
                    .agg(count($"uid").as("countVisits")).orderBy(desc("countVisits"), asc("domain")).limit(1000)
                    .orderBy(asc("domain"))
                    .select("domain").as[String].collect

Vocabulary = Array(.kasparov.ru, 0629.com.ua, 1001eda.com, 1001golos.ru, 1001goroskop.ru, 1001tur.ru, 101.ru, 11x11.ru, 123greetings.com, 123magazin.ru, 1prime.ru, 2012god.ru, 2015godkozy.com, 24au.ru, 24open.ru, 2ip.ru, 360.ru, 3d-galleru.ru, 3dnews.ru, 4tololo.ru, 5lb.ru, 62.ua, 7ba.ru, 7d.ru, 7days.ru, 9111.ru, 999.md, a-piter.ru, a.napishem.com, abc-people.com, account.forex4you.org, accuweather.com, actuallno.com, ad.adriver.ru, adidas.ru, adme.ru, adultmanga.ru, adultmult.tv, afisha.1ru.tv, aftershock.su, aif.ru, aif.ua, akusherstvo.ru, alfabank.ru, allday2.com, alleng.ru, allrecipes.ru, alpindustria.ru, amalgama-lab.com, amdm.ru, amic.ru, amoory.com, amway.ru, anekdot.ru, anekdotov.net, anistar.ru, anna-news.info, anysex.com, api.oktools.ru, app.facebook.com, apps....


Array(.kasparov.ru, 0629.com.ua, 1001eda.com, 1001golos.ru, 1001goroskop.ru, 1001tur.ru, 101.ru, 11x11.ru, 123greetings.com, 123magazin.ru, 1prime.ru, 2012god.ru, 2015godkozy.com, 24au.ru, 24open.ru, 2ip.ru, 360.ru, 3d-galleru.ru, 3dnews.ru, 4tololo.ru, 5lb.ru, 62.ua, 7ba.ru, 7d.ru, 7days.ru, 9111.ru, 999.md, a-piter.ru, a.napishem.com, abc-people.com, account.forex4you.org, accuweather.com, actuallno.com, ad.adriver.ru, adidas.ru, adme.ru, adultmanga.ru, adultmult.tv, afisha.1ru.tv, aftershock.su, aif.ru, aif.ua, akusherstvo.ru, alfabank.ru, allday2.com, alleng.ru, allrecipes.ru, alpindustria.ru, amalgama-lab.com, amdm.ru, amic.ru, amoory.com, amway.ru, anekdot.ru, anekdotov.net, anistar.ru, anna-news.info, anysex.com, api.oktools.ru, app.facebook.com, apps....

In [18]:
val dfForVector = dfLogsDomain.groupBy("uid").agg(collect_list("domain").as("domains"))

dfForVector = [uid: string, domains: array<string>]


[uid: string, domains: array<string>]

In [19]:
dfForVector.show(5)

+--------------------+--------------------+
|                 uid|             domains|
+--------------------+--------------------+
|0392f398-ea7e-4a1...|[adme.ru, adme.ru...|
|094b1e7e-97a6-441...|[kinoclips.net, n...|
|095544a2-64f7-422...|[well-good.ru, we...|
|098a0e00-8597-475...|[youtube.com, jke...|
|0a595fa1-bae0-41d...|[go.mail.ru, go.m...|
+--------------------+--------------------+
only showing top 5 rows



In [26]:
val cvm = new CountVectorizerModel(Vocabulary)
                .setInputCol("domains")
                .setOutputCol("domain_features")



cvm = cntVecModel_078e4efcf38a


cntVecModel_078e4efcf38a

In [28]:
val dfVector = cvm.transform(dfForVector)
                .withColumn("domain_features", asArray($"domain_features")).drop("domains")

dfVector = [uid: string, domain_features: array<double>]


[uid: string, domain_features: array<double>]

In [30]:
val dfPart1 = dfWorkOrNot
                .join(dfDayOfWeek, Seq("uid"), "inner")
                .join(dfHour, Seq("uid"), "inner")
                .join(dfVector, Seq("uid"), "inner")



dfPart1 = [uid: string, web_fraction_work_hours: double ... 33 more fields]


[uid: string, web_fraction_work_hours: double ... 33 more fields]

In [31]:
val dfFinal = dfPart1.join(dfUsersItems, Seq("uid"), "outer")

dfFinal = [uid: string, web_fraction_work_hours: double ... 673 more fields]


[uid: string, web_fraction_work_hours: double ... 673 more fields]

In [ ]:
dfFinal.write.mode("overwrite").parquet(outputPath)

In [ ]:
spark